# COMB FILTERING EXAMPLE
## a.k.a. What happens when you have 2 loudspeakers or microphones near each other
In order to use this notebook, Choose "Cell -> Run all".  Then scroll to the bottom and play around with the sliders

### First, import everything we need

In [4]:
import numpy as np
import scipy as sp
from ipywidgets import interact
from bokeh.io import curdoc, push_notebook, show, output_notebook
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d
from bokeh.models.widgets import Slider, TextInput
output_notebook()


Loading BokehJS ...

### Define the functions we'll be using

In [5]:
# Set up a couple globals
num_freqs = 5000
freqs  = np.logspace(np.log10(20), np.log10(20000), num_freqs, endpoint=True, base=10.0)
c = 343000.0 # mm/sec 

# Define functions for computing responses


def compute_distances(a, b):
    """a and b are 2d vecotrs of shape (n, 3) where n is the number of items, and 3 is for xyz dimensions                                                                                                          
    So, if we have a = (n, 3) and b = (m, 3), then the output will contain                                                                                                                                         
    an array of shape (n, m) and each element will be the distance from item a[x], b[y]"""
    aa = a[np.newaxis, :, :]
    bb = b[:, np.newaxis, :]
    ss = (aa - bb)**2
    s = np.sqrt(np.sum(ss, axis=2))
    return s

def compute_response(listener_distance, angles, speaker_locations, freqs):
    """This function looks at how far apart two speakers (or microphones) are, how far the listener is
    (listener_distance), and returns the frequency response"""
    angles_rad = np.deg2rad(angles)
    listener_positions = np.zeros((len(angles), 3))
    listener_positions[:, 0] = np.cos(angles_rad)
    listener_positions[:, 1] = np.sin(angles_rad)
    listeners = listener_positions * listener_distance
    speakers = np.array(speaker_locations)

    distances = compute_distances(speakers, listeners)  # mm                                                                                                                                                       
    # Consider distances[0, 0] to be amplitude = 1.0, and amplitude drops as 1/r**2                                                                                                                                
    amplitudes = 1.0/(distances / distances[0, 0])**2
    distances = distances[np.newaxis, :, :]
    freqs1 = freqs[:, np.newaxis, np.newaxis]
    phase = freqs1 * distances / c * 2 * np.pi   # cycles/sec * mm * sec / mm * rad/cycle = rad                                                                                                                    
    phasor = np.cos(phase) + 1j*np.sin(phase)
    amplitudes = amplitudes[np.newaxis, :, :]
    mag = np.abs(np.sum(phasor * amplitudes, axis=2))
    mag = mag / len(speakers)
    return mag, listeners, speakers

def update_data(speaker_distance = 30.0, listener_distance=1000.0, listener_angle=45.0):
    """update_data:  this function calls compute_response every time a slider is moved and 
    it updates the plots"""
    # Get current values                                                                                                                                                                                           
    sl = np.array([[0.0,  speaker_distance/2., 0],
                   [0.0, -speaker_distance/2., 0]])
    mag, listeners, speakers = compute_response(
        listener_distance = listener_distance,
        angles            = np.array([listener_angle], dtype=float),
        speaker_locations = sl,
        freqs = freqs)
    x = freqs
    y = 20*np.log10(np.abs(mag[:,0]))
    source.data = dict(x = x, y = y)
    graphic_source_speakers.data = dict(x = speakers[:, 0], y = speakers[:, 1])
    graphic_source_listeners.data = dict(x=[listeners[0, 0],], y=[listeners[0, 1],])
    x = np.transpose(np.array([speakers[:,0], np.tile(listeners[:,0], speakers.shape[0])]))
    y = np.transpose(np.array([speakers[:,1], np.tile(listeners[:,1], speakers.shape[0])]))
    graphic_source_lines.data = dict(x = x, y = y)
    push_notebook()


def create_plots():
    # Create data sources.  start with them empty  Sorry these are global... icky I know...
    source = ColumnDataSource(data = dict(x=[0.0], y= [0.0]))
    graphic_source_speakers = ColumnDataSource(data = dict(x=[0], y= [0]))
    graphic_source_listeners = ColumnDataSource(data = dict(x=[0], y=[0]))
    graphic_source_lines = ColumnDataSource(data = dict(x=[0], y=[0]))

    plot = figure(plot_width=400, 
                plot_height=400,
                title      = "Interference between sources",
                tools      = "crosshair,pan, reset, save, wheel_zoom",
                x_range    = [20, 20000],
                y_range    = [-60, 10])

    plot.yaxis.axis_label="Response (dB)"
    plot.xaxis.axis_label="Frequency"
    graphic = figure(plot_height= 400,
                     plot_width = 400,
                     title      = "Source Locations",
                     tools      = "crosshair,pan, reset, save, wheel_zoom",
                     x_range=[-1100, 1100],
                     y_range=[-1100, 1100],
                  )
    graphic.yaxis.axis_label="y (mm)"
    graphic.xaxis.axis_label="x (mm)"
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)
    graphic.annular_wedge(x='x', y='y', source = graphic_source_speakers, inner_radius=3, outer_radius = 30, start_angle=np.pi/2, end_angle=3*np.pi/2, color="green", alpha = 0.6)
    graphic.circle('x', 'y', source = graphic_source_listeners)
    graphic.multi_line('x', 'y', source=graphic_source_lines)

    return plot, graphic, source, graphic_source_speakers, graphic_source_listeners, graphic_source_lines


### And, finally do some plots
On the left-hand plot below (Source Locations), the two green thingies are loudspeakers.  All units are in millimeters.  The blue dot is the listener (i.e. your ear).  As you drag the sliders around, you can see that when the speakers are some distance apart, they start interfering with each other when off axis.  When on-axis, the response is always perfectly flat.    Interference is always worst when at 90 degrees off axis.

In [6]:

# Create the plots, and their respective data sources
plot, graphic, source, graphic_source_speakers, graphic_source_listeners, graphic_source_lines = create_plots()

# Show the plots
t = show(row(plot, graphic), notebook_handle=True)

# Create the interactive sliders
i = interact(update_data, 
         speaker_distance=(1.0, 1000.0), 
         listener_distance = (1.0, 2000.0),
         listener_angle = (0, 360, 1))